In [1]:
using HypothesisTests
using Distributions
using KernelDensity
using Optim
using Interpolations
using StatsBase

using PyPlot 

## SNe 

In [13]:
starting_pwd = pwd()

"/Users/nicholassenno/DataScience/sne_nu/notebooks"

In [17]:
#import some functions and constants to help with cosmology calculations
cd("../code")
include("cosmology.jl")

calc_D_prop (generic function with 1 method)

In [19]:
# Create an immutable type nu which contains the arrival time and direction (Right Assension RA and Declination Dec)
# These objects will be useful when creating random samples 

immutable nu
    mjd::Float64
    ang_err::Float64
    ra::Float64
    dec::Float64
end

# TODO : See if zenith_bin is necessary if we don't consider energy 

# custom type that will store all the relevant information for a SN, including its max time, location, 
# distance (redshift), and the neutrinos associated in its window

type sn
  max_date::Float64
  ra::Float64
  dec::Float64
  z::Float64
  zenith_bin::Int
  nb::Float64
  associated_nus::Array{nu,1}
  coefs::Array{Float64,1}

  function sn(max_date::Real,ra::Float64,dec::Float64,z::Float64)
      new_sn = new(max_date,ra,dec,z);
      zenith_bin = -1;
      new_sn.associated_nus=nu[];
      new_sn.coefs =Float64[];

      return new_sn
  end
end